In [155]:

import pymongo
from pymongo import MongoClient
import pandas as pd
import os
import bson
import json

#os.chdir('/Users/shannontran/Downloads') #changing directory to where my files are

In [4]:
# Connecting to Mongo db

In [284]:
client = MongoClient('localhost', 27017)

In [285]:
db = client.proj2_mongo
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'proj2_mongo')

In [286]:
# Importing datasets

In [287]:
# students.csv
students_df = pd.read_csv('students.csv')
students_df = students_df.iloc[1:] #removes first row with datatypes
students_df.head()

students_df = students_df.astype({'student_id':int, 'total_credits':int}) #changing data types from str to int


In [288]:
# in clubs 
in_club_df = pd.read_csv('in_club.csv')[1:]
in_club_df = in_club_df.astype({'student_id':int, 'club_id':int})

In [289]:
# clubs
club_df = pd.read_csv('club.csv')[1:]
club_df = club_df.astype({'club_id':int, 'num_members':int})

In [290]:
# department
dept_df = pd.read_csv('department.csv')[1:]
dept_df = dept_df.astype({'num_students':int, 'num_staff':int, 'budget':int})

In [291]:
# taking, use the taking/taken csv
taken_df = pd.read_csv('taken.csv')[1:]
taken_df = taken_df.astype({'student_id':int, 'year':int})

In [292]:
# Converting to dict
students_dict = students_df.to_dict('records')
in_club_dict = in_club_df.to_dict('records')
club_dict = club_df.to_dict('records')
dept_dict = dept_df.to_dict('records')
taken_dict = taken_df.to_dict('records')

In [293]:
# NESTING

In [294]:
# Nests club into in_club
for i in range(0, len(in_club_dict)):
    club_id = in_club_dict[i]['club_id']
    #print(club_id)
    #print(club_df[club_df['club_id'] == club_id])
    club_info = club_df[club_df['club_id'] == club_id][['club_name']].iloc[0].to_dict()
    in_club_dict[i]['club_info'] = club_info

#in_club_dict


In [295]:
# Nests student into in_club
for i in range(0, len(in_club_dict)):
    student_id = in_club_dict[i]['student_id']
    student_info = students_df[students_df['student_id'] == student_id][['first_name', 'last_name']].iloc[0].to_dict()
    in_club_dict[i]['student_info'] = student_info
in_club_dict

[{'student_id': 3,
  'club_id': 4,
  'role': 'President',
  'club_info': {'club_name': 'Club Running'},
  'student_info': {'first_name': 'Mary', 'last_name': 'Jane'}},
 {'student_id': 10,
  'club_id': 4,
  'role': 'Member',
  'club_info': {'club_name': 'Club Running'},
  'student_info': {'first_name': 'Forge', 'last_name': 'Launch'}},
 {'student_id': 5,
  'club_id': 4,
  'role': 'Member',
  'club_info': {'club_name': 'Club Running'},
  'student_info': {'first_name': 'Jasmine', 'last_name': 'Wang'}},
 {'student_id': 1,
  'club_id': 6,
  'role': 'Member',
  'club_info': {'club_name': 'Alternative Spring Break'},
  'student_info': {'first_name': 'John', 'last_name': 'Doe'}},
 {'student_id': 7,
  'club_id': 6,
  'role': 'Vice President',
  'club_info': {'club_name': 'Alternative Spring Break'},
  'student_info': {'first_name': 'Johnny', 'last_name': 'Depp'}},
 {'student_id': 10,
  'club_id': 1,
  'role': 'Member',
  'club_info': {'club_name': 'The Math Wizards'},
  'student_info': {'first_

In [296]:
# Nest courses taken into students
for i in range(0, len(students_dict)):
    student_id=students_dict[i]['student_id']
    class_info = taken_df[taken_df['student_id'] == student_id][['course_id', 'semester', 'year']]
    #print(class_info.iloc[:])
    if (False==class_info.empty):
        inf = class_info.to_dict()
    else:
        inf = class_info.to_dict()
    students_dict[i]['course_info'] = inf
students_dict   

[{'student_id': 1,
  'first_name': 'John',
  'last_name': 'Doe',
  'dept_code': 'ECON',
  'total_credits': 100,
  'student_year': 'Senior',
  'living_status': 'Off-campus',
  'course_info': {'course_id': {1: 'ECON125',
    2: 'IT214',
    3: 'MATH231',
    4: 'MATH232',
    5: 'MUSI144'},
   'semester': {1: 'Spring', 2: 'Spring', 3: 'Fall', 4: 'Spring', 5: 'Spring'},
   'year': {1: 2022, 2: 2022, 3: 2021, 4: 2022, 5: 2022}}},
 {'student_id': 2,
  'first_name': 'Jon',
  'last_name': 'Kashmeeri',
  'dept_code': 'BUS',
  'total_credits': 113,
  'student_year': 'Senior',
  'living_status': 'Off-campus',
  'course_info': {'course_id': {6: 'PSYC266', 7: 'STAT472', 8: 'LFIT115'},
   'semester': {6: 'Fall', 7: 'Spring', 8: 'Spring'},
   'year': {6: 2021, 7: 2022, 8: 2022}}},
 {'student_id': 3,
  'first_name': 'Mary',
  'last_name': 'Jane',
  'dept_code': 'NUTR',
  'total_credits': 25,
  'student_year': 'Freshman',
  'living_status': 'On-campus',
  'course_info': {'course_id': {9: 'ECON420', 10

In [297]:
students_dict
a=json.dumps(students_dict)
students_dict=json.loads(a)


In [298]:
# Connecting to MongoDB

In [299]:
in_club_collection = db.in_club # create collection

In [301]:
in_club_collection.delete_many({}) # reset just in case
in_club_collection.insert_many(in_club_dict) #inserts nested dictionary into in_club collection

In [302]:
# Who are club presidents?
query1 = in_club_collection.find({'role':'President'}, {'_id':0}) # fields set to zero are removed from output (for cleaner look)

In [303]:
import pprint #pretty print
for e in query1:
    pprint.pprint(e)

{'club_id': 4,
 'club_info': {'club_name': 'Club Running'},
 'role': 'President',
 'student_id': 3,
 'student_info': {'first_name': 'Mary', 'last_name': 'Jane'}}
{'club_id': 1,
 'club_info': {'club_name': 'The Math Wizards'},
 'role': 'President',
 'student_id': 2,
 'student_info': {'first_name': 'Jon', 'last_name': 'Kashmeeri'}}
{'club_id': 2,
 'club_info': {'club_name': 'Computational Data Sciences'},
 'role': 'President',
 'student_id': 20,
 'student_info': {'first_name': 'Kennedy', 'last_name': 'Vorkin'}}


In [305]:
query2 = in_club_collection.find({'role':'Member'}) # fields set to zero are removed from output (for cleaner look)

for e in query2:
    pprint.pprint(e)

{'_id': ObjectId('6298e3ad83d88a23300cf13d'),
 'club_id': 4,
 'club_info': {'club_name': 'Club Running'},
 'role': 'Member',
 'student_id': 10,
 'student_info': {'first_name': 'Forge', 'last_name': 'Launch'}}
{'_id': ObjectId('6298e3ad83d88a23300cf13e'),
 'club_id': 4,
 'club_info': {'club_name': 'Club Running'},
 'role': 'Member',
 'student_id': 5,
 'student_info': {'first_name': 'Jasmine', 'last_name': 'Wang'}}
{'_id': ObjectId('6298e3ad83d88a23300cf13f'),
 'club_id': 6,
 'club_info': {'club_name': 'Alternative Spring Break'},
 'role': 'Member',
 'student_id': 1,
 'student_info': {'first_name': 'John', 'last_name': 'Doe'}}
{'_id': ObjectId('6298e3ad83d88a23300cf141'),
 'club_id': 1,
 'club_info': {'club_name': 'The Math Wizards'},
 'role': 'Member',
 'student_id': 10,
 'student_info': {'first_name': 'Forge', 'last_name': 'Launch'}}
{'_id': ObjectId('6298e3ad83d88a23300cf143'),
 'club_id': 1,
 'club_info': {'club_name': 'The Math Wizards'},
 'role': 'Member',
 'student_id': 1,
 'stude

In [306]:
# Who are seniors by credits? (>=90 credits)
students_collection = db.students
students_collection.delete_many({}) #reset just in case
students_collection.insert_many(students_dict)


In [308]:
query3 = students_collection.find({'total_credits':{'$gte':90}}, {'living_status':0, 'student_id':0, 'dept_code':0, 'course_info':0}) # fields set to zero are removed from output (for cleaner look)

for e in query3:
    pprint.pprint(e)

{'_id': ObjectId('6298e43183d88a23300cf14c'),
 'first_name': 'John',
 'last_name': 'Doe',
 'student_year': 'Senior',
 'total_credits': 100}
{'_id': ObjectId('6298e43183d88a23300cf14d'),
 'first_name': 'Jon',
 'last_name': 'Kashmeeri',
 'student_year': 'Senior',
 'total_credits': 113}
{'_id': ObjectId('6298e43183d88a23300cf14f'),
 'first_name': 'Mark',
 'last_name': 'Maguire',
 'student_year': 'Senior',
 'total_credits': 95}
{'_id': ObjectId('6298e43183d88a23300cf156'),
 'first_name': 'Silas',
 'last_name': 'French',
 'student_year': 'Senior',
 'total_credits': 90}
{'_id': ObjectId('6298e43183d88a23300cf157'),
 'first_name': 'Ronald',
 'last_name': 'McDonald',
 'student_year': 'Senior',
 'total_credits': 102}
{'_id': ObjectId('6298e43183d88a23300cf158'),
 'first_name': 'Walt',
 'last_name': 'White',
 'student_year': 'Senior',
 'total_credits': 120}
{'_id': ObjectId('6298e43183d88a23300cf15b'),
 'first_name': 'Bo',
 'last_name': 'Bo',
 'student_year': 'Senior',
 'total_credits': 92}
{'_i

In [309]:
dept_collection = db.dept
dept_collection.delete_many({})
dept_collection.insert_many(dept_dict)

In [311]:
#What are the top 5 most popular majors?
pipe = ([
     { "$sort" : { "num_students": -1 } },
     { "$limit": 5}
   ])

query4 = dept_collection.aggregate(pipe)
for e in query4:
    pprint.pprint(e)

{'_id': ObjectId('6298e45583d88a23300cf168'),
 'budget': 18000,
 'dept_code': 'LFIT',
 'dept_name': 'Fitness',
 'num_staff': 16,
 'num_students': 245}
{'_id': ObjectId('6298e45583d88a23300cf160'),
 'budget': 230000,
 'dept_code': 'BIOL',
 'dept_name': 'Biology',
 'num_staff': 108,
 'num_students': 223}
{'_id': ObjectId('6298e45583d88a23300cf165'),
 'budget': 16000,
 'dept_code': 'ENGL',
 'dept_name': 'English',
 'num_staff': 24,
 'num_students': 204}
{'_id': ObjectId('6298e45583d88a23300cf167'),
 'budget': 200000,
 'dept_code': 'IT',
 'dept_name': 'Information Technology',
 'num_staff': 46,
 'num_students': 200}
{'_id': ObjectId('6298e45583d88a23300cf163'),
 'budget': 190000,
 'dept_code': 'CS',
 'dept_name': 'Computer Science',
 'num_staff': 40,
 'num_students': 170}


In [312]:
#students_collection.delete_many({})

#What are the top 5 most funded departments?
pipe = ([


     { "$sort" : { "budget": -1 } },
     { "$limit": 5}
   ])

query5 = dept_collection.aggregate(pipe)
for e in query5:
    pprint.pprint(e)

{'_id': ObjectId('6298e45583d88a23300cf160'),
 'budget': 230000,
 'dept_code': 'BIOL',
 'dept_name': 'Biology',
 'num_staff': 108,
 'num_students': 223}
{'_id': ObjectId('6298e45583d88a23300cf167'),
 'budget': 200000,
 'dept_code': 'IT',
 'dept_name': 'Information Technology',
 'num_staff': 46,
 'num_students': 200}
{'_id': ObjectId('6298e45583d88a23300cf164'),
 'budget': 200000,
 'dept_code': 'ECON',
 'dept_name': 'Economics',
 'num_staff': 50,
 'num_students': 150}
{'_id': ObjectId('6298e45583d88a23300cf161'),
 'budget': 200000,
 'dept_code': 'BUS',
 'dept_name': 'Business',
 'num_staff': 45,
 'num_students': 143}
{'_id': ObjectId('6298e45583d88a23300cf163'),
 'budget': 190000,
 'dept_code': 'CS',
 'dept_name': 'Computer Science',
 'num_staff': 40,
 'num_students': 170}


In [317]:
#in_club_collection.delete_many({})
#Which semester had the most courses / sections offered?

students_collection = 



In [39]:
dept_dict

[{'dept_code': 'BIOL',
  'dept_name': 'Biology',
  'num_staff': 108,
  'num_students': 223,
  'budget': 230000,
  '_id': ObjectId('6298b79183d88a23300cf0b3')},
 {'dept_code': 'BUS',
  'dept_name': 'Business',
  'num_staff': 45,
  'num_students': 143,
  'budget': 200000,
  '_id': ObjectId('6298b79183d88a23300cf0b4')},
 {'dept_code': 'CHEM',
  'dept_name': 'Chemistry',
  'num_staff': 25,
  'num_students': 86,
  'budget': 120000,
  '_id': ObjectId('6298b79183d88a23300cf0b5')},
 {'dept_code': 'CS',
  'dept_name': 'Computer Science',
  'num_staff': 40,
  'num_students': 170,
  'budget': 190000,
  '_id': ObjectId('6298b79183d88a23300cf0b6')},
 {'dept_code': 'ECON',
  'dept_name': 'Economics',
  'num_staff': 50,
  'num_students': 150,
  'budget': 200000,
  '_id': ObjectId('6298b79183d88a23300cf0b7')},
 {'dept_code': 'ENGL',
  'dept_name': 'English',
  'num_staff': 24,
  'num_students': 204,
  'budget': 16000,
  '_id': ObjectId('6298b79183d88a23300cf0b8')},
 {'dept_code': 'HIST',
  'dept_name'